# Prepare experimentation

In [1]:
EXPERIMENT_ID = "exp004"
TRAIN_DATA_FILE = "artifacts/data/all_train.csv"
BACKTESTING_STRATEGY = "train-val"
TEST_SIZE_RATIO = 0.2
BACKTESTING_VAL_RATIO = 0.1
BACKTESTING_PERF_RATIO = 0.1
FINAL_VAL_RATIO = 0.1
TARGET_COLUMN = "Sentiment"

## Generate splits

In [ ]:
!uv run python main.py generate-splits \
    --experiment-id $EXPERIMENT_ID \
    --input-file $TRAIN_DATA_FILE \
    --backtesting-strategy $BACKTESTING_STRATEGY \
    --test-size $TEST_SIZE_RATIO \
    --backtesting-val-size $BACKTESTING_VAL_RATIO \
    --perf-estimation-val-size $BACKTESTING_PERF_RATIO \
    --final-model-val-size $FINAL_VAL_RATIO

!ls -lah artifacts/experiments/exp004
!ls -lah artifacts/experiments/exp004/backtesting
!ls -lah artifacts/experiments/exp004/performance_estimation/
!ls -lah artifacts/experiments/exp004/final_model

# Develop models...

In [ ]:
!uv run python main.py list-models

# Run experiments

## SVM base

In [ ]:
SVM_BASE_CFG = "svm-base"

In [ ]:
!uv run python main.py run-backtesting \
    --experiment-id $EXPERIMENT_ID \
    --model-config-name $SVM_BASE_CFG

## SVM with hiperparameter optimization

In [ ]:
SVM_OPT_CFG = "svm-opt"

In [ ]:
!uv run python main.py run-backtesting \
    --experiment-id $EXPERIMENT_ID \
    --model-config-name $SVM_OPT_CFG

## First check - check our baseline performance

In [4]:
!uv run python main.py compare-models \
    --experiment-id $EXPERIMENT_ID

2025-10-18 12:49:22.593 | WARNING  | evaluate:compare_models:56 - No results file found for run type 'performance_estimation' in experiment 'exp004'.
2025-10-18 12:49:22.593 | INFO     | evaluate:compare_models:79 - --- Model Comparison for Experiment 'exp004' ---
2025-10-18 12:49:22.594 | INFO     | evaluate:compare_models:82 - 
--- Run Type: backtesting ---
2025-10-18 12:49:22.594 | INFO     | evaluate:compare_models:91 - Summary for backtesting (train-val strategy):
2025-10-18 12:49:22.597 | INFO     | evaluate:compare_models:95 - 
  model_name  accuracy precision    recall  f1_score
                  mean      mean      mean      mean
0   svm-base  0.619658  0.627262  0.619658  0.621129
1    svm-opt  0.692308  0.678169  0.692308  0.674845


In [6]:
import pandas as pd
import io
data = """mean      mean      mean      mean
    0   svm-base  0.619658  0.627262  0.619658  0.621129
    1    svm-opt  0.692308  0.678169  0.692308  0.674845
    """
df = pd.read_csv(
        io.StringIO(data),
        skiprows=1,
        delim_whitespace=True,
        header=None,
        index_col=0,
        names=['model', 'accuracy', 'precision', 'recall', 'f1_score']
   )
df

/var/folders/fv/4xmxw7lx1l5d8bj50hs13zl00000gp/T/ipykernel_79055/912242006.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


,model,accuracy,precision,recall,f1_score
0,svm-base,0.619658,0.627262,0.619658,0.621129
1,svm-opt,0.692308,0.678169,0.692308,0.674845


## Transformer model (Roberta)

### Micro-bert

[HuggingFace Link](https://huggingface.co/boltuix/bert-micro)

In [7]:
MICRO_BERT_BASE = "bert-micro-base"

In [11]:
!uv run python main.py run-backtesting \
    --experiment-id $EXPERIMENT_ID \
    --model-config-name $MICRO_BERT_BASE

2025-10-18 12:58:44.187 | INFO     | mlops.runner:run_backtesting:43 - --- Backtesting Fold 0 ---
2025-10-18 12:58:44.188 | INFO     | mlops.runner:load_model_from_config:24 - Loading model: RobertaModel from models.roberta.model
2025-10-18 12:59:00.229 | INFO     | mlops.runner:run_backtesting:54 - Training model...
2025-10-18 12:59:00.229 | INFO     | models.roberta.pipeline:run:15 - Preprocessing data for RoBERTa model...
Map: 100%|█████████████████████████| 4205/4205 [00:00<00:00, 9028.66 examples/s]
2025-10-18 12:59:01.345 | INFO     | models.roberta.pipeline:run:41 - Preprocessing validation data...
Map: 100%|██████████████████████████| 468/468 [00:00<00:00, 10132.69 examples/s]
2025-10-18 12:59:01.399 | INFO     | models.roberta.pipeline:run:48 - Validation data preprocessing complete.
2025-10-18 12:59:01.399 | INFO     | models.roberta.pipeline:run:53 - Data preprocessing complete.
2025-10-18 12:59:01.399 | INFO     | models.roberta.pipeline:run:58 - Training RoBERTa model...
S

## Second check

In [24]:
!uv run python main.py compare-models \
    --experiment-id $EXPERIMENT_ID

2025-10-18 13:08:06.179 | WARNING  | evaluate:compare_models:56 - No results file found for run type 'performance_estimation' in experiment 'exp004'.
2025-10-18 13:08:06.180 | INFO     | evaluate:compare_models:79 - --- Model Comparison for Experiment 'exp004' ---
2025-10-18 13:08:06.180 | INFO     | evaluate:compare_models:82 - 
--- Run Type: backtesting ---
2025-10-18 13:08:06.180 | INFO     | evaluate:compare_models:91 - Summary for backtesting (train-val strategy):
2025-10-18 13:08:06.183 | INFO     | evaluate:compare_models:95 - 
        model_name  accuracy precision    recall  f1_score
                        mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1         svm-base  0.619658  0.627262  0.619658  0.621129
2          svm-opt  0.692308  0.678169  0.692308  0.674845


In [9]:
data = """mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1         svm-base  0.619658  0.627262  0.619658  0.621129
2          svm-opt  0.692308  0.678169  0.692308  0.674845
"""
df = pd.read_csv(
        io.StringIO(data),
        skiprows=1,
        delim_whitespace=True,
        header=None,
        index_col=0,
        names=['model', 'accuracy', 'precision', 'recall', 'f1_score']
   )
df

/var/folders/fv/4xmxw7lx1l5d8bj50hs13zl00000gp/T/ipykernel_79055/3415334600.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


,model,accuracy,precision,recall,f1_score
0,bert-micro-base,0.536325,0.287644,0.536325,0.374458
1,svm-base,0.619658,0.627262,0.619658,0.621129
2,svm-opt,0.692308,0.678169,0.692308,0.674845


Big dissapointment! Let's check more advanced model.

## Optimized mirco-bert 

In [25]:
MICRO_BERT_LONG = "bert-micro-long"

In [2]:
!uv run python main.py run-backtesting \
    --experiment-id $EXPERIMENT_ID \
    --model-config-name $MICRO_BERT_LONG

Usage: main.py run-backtesting [OPTIONS]
Try 'main.py run-backtesting --help' for help.
╭─ Error ──────────────────────────────────────────────────────────────────────╮
│ Missing option '--model-config-name'.                                        │
╰──────────────────────────────────────────────────────────────────────────────╯


## Third check

In [3]:
!uv run python main.py compare-models \
    --experiment-id $EXPERIMENT_ID

2025-10-18 13:32:52.877 | WARNING  | evaluate:compare_models:56 - No results file found for run type 'performance_estimation' in experiment 'exp004'.
2025-10-18 13:32:52.878 | INFO     | evaluate:compare_models:79 - --- Model Comparison for Experiment 'exp004' ---
2025-10-18 13:32:52.879 | INFO     | evaluate:compare_models:82 - 
--- Run Type: backtesting ---
2025-10-18 13:32:52.879 | INFO     | evaluate:compare_models:91 - Summary for backtesting (train-val strategy):
2025-10-18 13:32:52.883 | INFO     | evaluate:compare_models:95 - 
        model_name  accuracy precision    recall  f1_score
                        mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1         svm-base  0.619658  0.627262  0.619658  0.621129
2          svm-opt  0.692308  0.678169  0.692308  0.674845


In [4]:
data = """model_name  accuracy precision    recall  f1_score
                        mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1         svm-base  0.619658  0.627262  0.619658  0.621129
2          svm-opt  0.692308  0.678169  0.692308  0.674845
"""
df = pd.read_csv(
        io.StringIO(data),
        skiprows=1,
        delim_whitespace=True,
        header=None,
        index_col=0,
        names=['model', 'accuracy', 'precision', 'recall', 'f1_score']
   )
df

NameError: name 'pd' is not defined

## Bigger model Roberta

In [ ]:
ROBERTA_BASE = "roberta-base"


In [ ]:
!uv run python main.py run-backtesting \
    --experiment-id $EXPERIMENT_ID\
    --model-config-name $ROBERTA_BASE

2025-10-18 16:28:15.714 | INFO     | mlops.runner:run_backtesting:58 - --- Backtesting Fold 0 ---
2025-10-18 16:28:15.715 | INFO     | mlops.runner:_load_model_from_config:28 - Loading model: RobertaModel from models.roberta.model
2025-10-18 16:28:19.662 | INFO     | mlops.runner:run_backtesting:68 - Training model...
2025-10-18 16:28:19.662 | INFO     | models.roberta.pipeline:run:15 - Preprocessing data for RoBERTa model...
Map: 100%|████████████████████████| 4205/4205 [00:00<00:00, 11992.09 examples/s]
2025-10-18 16:28:20.650 | INFO     | models.roberta.pipeline:run:41 - Preprocessing validation data...
Map: 100%|██████████████████████████| 468/468 [00:00<00:00, 12096.20 examples/s]
2025-10-18 16:28:20.710 | INFO     | models.roberta.pipeline:run:48 - Validation data preprocessing complete.
2025-10-18 16:28:20.710 | INFO     | models.roberta.pipeline:run:53 - Data preprocessing complete.
2025-10-18 16:28:20.710 | INFO     | models.roberta.pipeline:run:58 - Training RoBERTa model...


## Fourth check

In [14]:
!uv run python main.py compare-models \
    --experiment-id $EXPERIMENT_ID

2025-10-18 16:27:53.684 | WARNING  | evaluate:compare_models:56 - No results file found for run type 'performance_estimation' in experiment 'exp004'.
2025-10-18 16:27:53.685 | INFO     | evaluate:compare_models:79 - --- Model Comparison for Experiment 'exp004' ---
2025-10-18 16:27:53.685 | INFO     | evaluate:compare_models:82 - 
--- Run Type: backtesting ---
2025-10-18 16:27:53.685 | INFO     | evaluate:compare_models:91 - Summary for backtesting (train-val strategy):
2025-10-18 16:27:53.690 | INFO     | evaluate:compare_models:95 - 
        model_name  accuracy precision    recall  f1_score
                        mean      mean      mean      mean
0  bert-micro-base  0.536325  0.287644  0.536325  0.374458
1         svm-base  0.619658  0.627262  0.619658  0.621129
2          svm-opt  0.692308  0.678169  0.692308  0.674845
